# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fef3059b2b0>
├── 'a' --> tensor([[ 0.8213,  0.6211, -0.8185],
│                   [ 1.1080, -0.8684,  0.8007]])
└── 'x' --> <FastTreeValue 0x7fef30a9b370>
    └── 'c' --> tensor([[ 1.2208, -1.0106, -0.9093,  0.2586],
                        [ 1.1952,  0.3686,  1.3474, -1.0343],
                        [-0.1250,  1.0669, -0.9628, -2.0326]])

In [4]:
t.a

tensor([[ 0.8213,  0.6211, -0.8185],
        [ 1.1080, -0.8684,  0.8007]])

In [5]:
%timeit t.a

61.9 ns ± 0.0104 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fef3059b2b0>
├── 'a' --> tensor([[ 0.6683,  0.0933, -0.4611],
│                   [ 2.8841, -2.4396,  0.5224]])
└── 'x' --> <FastTreeValue 0x7fef30a9b370>
    └── 'c' --> tensor([[ 1.2208, -1.0106, -0.9093,  0.2586],
                        [ 1.1952,  0.3686,  1.3474, -1.0343],
                        [-0.1250,  1.0669, -0.9628, -2.0326]])

In [7]:
%timeit t.a = new_value

62.6 ns ± 0.0161 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8213,  0.6211, -0.8185],
               [ 1.1080, -0.8684,  0.8007]]),
    x: Batch(
           c: tensor([[ 1.2208, -1.0106, -0.9093,  0.2586],
                      [ 1.1952,  0.3686,  1.3474, -1.0343],
                      [-0.1250,  1.0669, -0.9628, -2.0326]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8213,  0.6211, -0.8185],
        [ 1.1080, -0.8684,  0.8007]])

In [11]:
%timeit b.a

58.1 ns ± 0.00879 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.9298,  1.6962, -0.0112],
               [ 0.6373, -0.1859,  0.7495]]),
    x: Batch(
           c: tensor([[ 1.2208, -1.0106, -0.9093,  0.2586],
                      [ 1.1952,  0.3686,  1.3474, -1.0343],
                      [-0.1250,  1.0669, -0.9628, -2.0326]]),
       ),
)

In [13]:
%timeit b.a = new_value

477 ns ± 0.119 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

828 ns ± 0.361 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 70 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 303 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 617 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fef1ecc88e0>
├── 'a' --> tensor([[[ 0.8213,  0.6211, -0.8185],
│                    [ 1.1080, -0.8684,  0.8007]],
│           
│                   [[ 0.8213,  0.6211, -0.8185],
│                    [ 1.1080, -0.8684,  0.8007]],
│           
│                   [[ 0.8213,  0.6211, -0.8185],
│                    [ 1.1080, -0.8684,  0.8007]],
│           
│                   [[ 0.8213,  0.6211, -0.8185],
│                    [ 1.1080, -0.8684,  0.8007]],
│           
│                   [[ 0.8213,  0.6211, -0.8185],
│                    [ 1.1080, -0.8684,  0.8007]],
│           
│                   [[ 0.8213,  0.6211, -0.8185],
│                    [ 1.1080, -0.8684,  0.8007]],
│           
│                   [[ 0.8213,  0.6211, -0.8185],
│                    [ 1.1080, -0.8684,  0.8007]],
│           
│                   [[ 0.8213,  0.6211, -0.8185],
│                    [ 1.1080, -0.8684,  0.8007]]])
└── 'x' --> <FastTreeValue 0x7fef1ecc8f70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 39.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fef1f293910>
├── 'a' --> tensor([[ 0.8213,  0.6211, -0.8185],
│                   [ 1.1080, -0.8684,  0.8007],
│                   [ 0.8213,  0.6211, -0.8185],
│                   [ 1.1080, -0.8684,  0.8007],
│                   [ 0.8213,  0.6211, -0.8185],
│                   [ 1.1080, -0.8684,  0.8007],
│                   [ 0.8213,  0.6211, -0.8185],
│                   [ 1.1080, -0.8684,  0.8007],
│                   [ 0.8213,  0.6211, -0.8185],
│                   [ 1.1080, -0.8684,  0.8007],
│                   [ 0.8213,  0.6211, -0.8185],
│                   [ 1.1080, -0.8684,  0.8007],
│                   [ 0.8213,  0.6211, -0.8185],
│                   [ 1.1080, -0.8684,  0.8007],
│                   [ 0.8213,  0.6211, -0.8185],
│                   [ 1.1080, -0.8684,  0.8007]])
└── 'x' --> <FastTreeValue 0x7fef1dc6ff40>
    └── 'c' --> tensor([[ 1.2208, -1.0106, -0.9093,  0.2586],
                        [ 1.1952,  0.3686,  1.3474, -1.0343],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 74.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58 µs ± 35 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.2208, -1.0106, -0.9093,  0.2586],
                       [ 1.1952,  0.3686,  1.3474, -1.0343],
                       [-0.1250,  1.0669, -0.9628, -2.0326]],
              
                      [[ 1.2208, -1.0106, -0.9093,  0.2586],
                       [ 1.1952,  0.3686,  1.3474, -1.0343],
                       [-0.1250,  1.0669, -0.9628, -2.0326]],
              
                      [[ 1.2208, -1.0106, -0.9093,  0.2586],
                       [ 1.1952,  0.3686,  1.3474, -1.0343],
                       [-0.1250,  1.0669, -0.9628, -2.0326]],
              
                      [[ 1.2208, -1.0106, -0.9093,  0.2586],
                       [ 1.1952,  0.3686,  1.3474, -1.0343],
                       [-0.1250,  1.0669, -0.9628, -2.0326]],
              
                      [[ 1.2208, -1.0106, -0.9093,  0.2586],
                       [ 1.1952,  0.3686,  1.3474, -1.0343],
                       [-0.1250,  1.0669, -0.9628, -2.0326]],

In [26]:
%timeit Batch.stack(batches)

77 µs ± 150 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.2208, -1.0106, -0.9093,  0.2586],
                      [ 1.1952,  0.3686,  1.3474, -1.0343],
                      [-0.1250,  1.0669, -0.9628, -2.0326],
                      [ 1.2208, -1.0106, -0.9093,  0.2586],
                      [ 1.1952,  0.3686,  1.3474, -1.0343],
                      [-0.1250,  1.0669, -0.9628, -2.0326],
                      [ 1.2208, -1.0106, -0.9093,  0.2586],
                      [ 1.1952,  0.3686,  1.3474, -1.0343],
                      [-0.1250,  1.0669, -0.9628, -2.0326],
                      [ 1.2208, -1.0106, -0.9093,  0.2586],
                      [ 1.1952,  0.3686,  1.3474, -1.0343],
                      [-0.1250,  1.0669, -0.9628, -2.0326],
                      [ 1.2208, -1.0106, -0.9093,  0.2586],
                      [ 1.1952,  0.3686,  1.3474, -1.0343],
                      [-0.1250,  1.0669, -0.9628, -2.0326],
                      [ 1.2208, -1.0106, -0.9093,  0.2586],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 375 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 6.64 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
